In [1]:
import osmnx as ox
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# osmnx.geometries.geometries_from_bbox(north, south, east, west, tags)
# https://www.mapsdirections.info/pt/coordenadas-gps.html
north = -6.9875
west = -37.3303
south = -7.0882
east = -37.2215
geoms = ox.geometries.geometries_from_bbox(north, south, east, west, tags={'amenity':True})

In [3]:
geoms.head()

amenity                        name  \
element_type osmid                                            
node         1798496605    fuel                 Posto Viola   
             1798496642    taxi  Moto táxi Praça dos Pombos   
             1798525769    taxi       Táxi Rodoviária Velha   
             1798525772    taxi    Táxi Fórum Miguel Sátiro   
             1798919196     bar                 Bar do Rock   

                                           geometry  \
element_type osmid                                    
node         1798496605  POINT (-37.27847 -7.02390)   
             1798496642  POINT (-37.27843 -7.02361)   
             1798525769  POINT (-37.27735 -7.02265)   
             1798525772  POINT (-37.27702 -7.02601)   
             1798919196  POINT (-37.27702 -7.02204)   

                                                      opening_hours  \
element_type osmid                                                    
node         1798496605                                         NaN   
             1798496642                                         NaN   
             1798525769                                         NaN   
             1798525772                                         NaN   
             1798919196  We-Su 19:00-02:00+; week 25-26 18:00-04:00   

                        outdoor_seating cuisine  atm brand brand:wikidata  \
element_type osmid                                                          
node         1798496605             NaN     NaN  NaN   NaN            NaN   
             1798496642             NaN     NaN  NaN   NaN            NaN   
             1798525769             NaN     NaN  NaN   NaN            NaN   
             1798525772             NaN     NaN  NaN   NaN            NaN   
             1798919196             yes     NaN  NaN   NaN            NaN   

                        brand:wikipedia  ... note park_ride shelter  \
element_type osmid                       ...                          
node         1798496605             NaN  ...  NaN       NaN     NaN   
             1798496642             NaN  ...  NaN       NaN     NaN   
             1798525769             NaN  ...  NaN       NaN     NaN   
             1798525772             NaN  ...  NaN       NaN     NaN   
             1798919196             NaN  ...  NaN       NaN     NaN   

                        roof:height roof:orientation  lit source:name ways  \
element_type osmid                                                           
node         1798496605         NaN              NaN  NaN         NaN  NaN   
             1798496642         NaN              NaN  NaN         NaN  NaN   
             1798525769         NaN              NaN  NaN         NaN  NaN   
             1798525772         NaN              NaN  NaN         NaN  NaN   
             1798919196         NaN              NaN  NaN         NaN  NaN   

                        layer type  
element_type osmid                  
node         1798496605   NaN  NaN  
             1798496642   NaN  NaN  
             1798525769   NaN  NaN  
             1798525772   NaN  NaN  
             1798919196   NaN  NaN  

[5 rows x 92 columns]

In [4]:
geoms = pd.DataFrame(geoms)
geoms.reset_index(inplace=True)
geoms.head()

,element_type,osmid,amenity,name,geometry,opening_hours,outdoor_seating,cuisine,atm,brand,...,note,park_ride,shelter,roof:height,roof:orientation,lit,source:name,ways,layer,type
0,node,1798496605,fuel,Posto Viola,POINT (-37.27847 -7.02390),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,1798496642,taxi,Moto táxi Praça dos Pombos,POINT (-37.27843 -7.02361),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node,1798525769,taxi,Táxi Rodoviária Velha,POINT (-37.27735 -7.02265),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,1798525772,taxi,Táxi Fórum Miguel Sátiro,POINT (-37.27702 -7.02601),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node,1798919196,bar,Bar do Rock,POINT (-37.27702 -7.02204),We-Su 19:00-02:00+; week 25-26 18:00-04:00,yes,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
geoms[geoms.geometry.apply(type).apply(str)=="<class 'shapely.geometry.polygon.Polygon'>"].head()

,element_type,osmid,amenity,name,geometry,opening_hours,outdoor_seating,cuisine,atm,brand,...,note,park_ride,shelter,roof:height,roof:orientation,lit,source:name,ways,layer,type
638,way,165773628,parking,NaN,"POLYGON ((-37.30354 -7.00531, -37.30316 -7.005...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
639,way,166259294,place_of_worship,NaN,"POLYGON ((-37.30145 -7.00646, -37.30139 -7.006...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
640,way,166259767,parking,NaN,"POLYGON ((-37.30149 -7.00651, -37.30200 -7.006...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
641,way,166429876,place_of_worship,Paróquia Nossa Senhora Das Neves,"POLYGON ((-37.30090 -7.03673, -37.30082 -7.037...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
642,way,166550089,college,IFPB - Campus Patos,"POLYGON ((-37.28455 -7.07763, -37.28672 -7.076...",Su-Fr 07:00-22:00; week 01 off; week 53 off; S...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
geoms.amenity.value_counts()

parking               96
fast_food             75
restaurant            69
bar                   68
bench                 67
place_of_worship      39
clinic                39
pharmacy              39
toilets               37
doctors               27
school                26
fuel                  25
taxi                  21
dentist               21
car_wash              17
telephone             14
ice_cream             12
university             9
hospital               9
bank                   8
pub                    7
police                 7
waste_basket           7
atm                    6
car_rental             5
drinking_water         5
marketplace            4
parking_entrance       4
cafe                   4
courthouse             4
community_centre       4
register_office        3
kindergarten           3
language_school        3
library                3
nightclub              3
post_office            3
bicycle_parking        2
prison                 2
bus_station            2


In [7]:
G = ox.graph_from_place('Centro, Patos, Paraíba, Brazil', network_type='drive')
G = ox.project_graph(G)

In [8]:
import matplotlib.pyplot as plt
import osmnx as ox
ox.config(use_cache=True, log_console=True)

G = ox.graph_from_address('Centro, Patos, Paraíba, Brazil', dist=200, network_type='drive')
G = ox.get_undirected(G)

fig, ax = ox.plot_graph(G, bgcolor='k', edge_linewidth=3, node_size=0,
                        show=False, close=False)
for _, edge in ox.graph_to_gdfs(G, nodes=False).fillna('').iterrows():
    c = edge['geometry'].centroid
    text = edge['name']
    ax.annotate(text, (c.x, c.y), c='w')
plt.show()

# python MY_SCRIPT.py >/dev/null &

/home/carlos/.local/lib/python3.9/site-packages/osmnx/utils.py:192: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  warnings.warn(


OSError: [Errno 5] Input/output error